### This file was used for extraction and transformation. It is used to create dimension and fact tables which is then loaded into the datawarehouse using the bulk query file. 

Step 1: Create database and tables using the SQL Files, create_database file, create_table_relations file in order.
Step 2. Run this file to obtain dimension and fact tables. Then run Bulk_Query sql file to populate tables.
Or use the pre-existing files in the csv files folder after step one. This script will create the same files as the csv files folder.Then run Bulk_Query sql file to populate tables.

### Data Reading and combining databases

In [2]:
import pandas as pd
df_seek = pd.read_csv('seek_australia.csv')
df_reed=pd.read_csv('reed_uk.csv')

In [3]:
combined_job_df = pd.concat([df_seek, df_reed], axis=0)

In [4]:
combined_job_df.tail()

,category,city,company_name,geo,job_board,job_description,job_title,job_type,post_date,salary_offered,state,url,job_requirements
49995,science jobs,Oxfordshire,Hays Specialist Recruitment Limited,uk,reed,Apply now Scientist - Cell Based Assays - Neu...,Scientist In vitro Cell-Free & Cell Based Assays,"Contract, full-time",3/7/2018,£22.00 - £26.00 per hour,South East England,NaN,NaN
49996,graduate training internships jobs,Northampton,The Graduate,uk,reed,Apply now Our client is looking for high achi...,Graduate Training Scheme - Recruitment,"Permanent, full-time",3/9/2018,"£18,000 - £21,000 per annum",Northamptonshire,NaN,Required skills Blue Chip Communication Skill...
49997,strategy consultancy jobs,City of London,Capgemini Consulting,uk,reed,Apply on employer's website Who youâ€™ll be w...,"Management Consultant - Retail Buying, Merchan...","Permanent, full-time",2/17/2018,Competitive salary,London,NaN,NaN
49998,purchasing jobs,Egham,Concept Human Solutions,uk,reed,"Apply now UNDERSTANDING, SERVICE, TRUST3 Word...",AV Procurement Assistant,"Permanent, full-time",3/6/2018,"£27,500 - £30,000 per annum, negotiable, OTE",Surrey,NaN,Required skills Procurement AV
49999,fmcg jobs,London,D R Newitt,uk,reed,"Apply now My client, highly successful, rapid...",Supply Planner - French Speaking,"Permanent, full-time",2/27/2018,"£45,000 per annum",South East England,NaN,Required skills FMCG French Supply Planning


## Data Cleaning of dimensions. 

In [5]:
newdf=combined_job_df.fillna("NOT LISTED")

#### Date Cleaning Function

In [6]:

## Grab the date part
def dateclean(str):
    if 'T' in str:
        new_str=str.replace(str,str.split('T')[0])
        str=str.replace(str,new_str)
    return str          

In [7]:
## New df contains combined seek and reed database.

newdf['post_date']=newdf['post_date'].apply(dateclean)
newdf['new_post_date']=pd.to_datetime(newdf['post_date'])
newdf['new_post_date']




0       2018-04-15
1       2018-04-15
2       2018-04-15
3       2018-04-16
4       2018-04-16
           ...    
49995   2018-03-07
49996   2018-03-09
49997   2018-02-17
49998   2018-03-06
49999   2018-02-27
Name: new_post_date, Length: 80000, dtype: datetime64[ns]

### Job Title cleaning function applied later

In [8]:
import re
def clean_titles(str):
    
    cha_lst=['-','/',',','+','@','(','â€“','&',"?",':','.','|','and','full','Full','part','Part']
    for char in cha_lst:
        if char in str:
            new_str=str.replace(str,str.split(char)[0])
            if len(new_str)>10:
                str=str.replace(str,new_str)   
    
    return re.sub(r'[^A-Za-z0-9 ]+','',str) 

#### Category cleaning function applied later

In [9]:
def clean_cat(str):
  
    if 'catering' in str.lower():
            str=str.replace(str,'Hospitality  Tourism')
    elif 'law' in str.lower():
            str=str.replace(str,'Legal')
    elif 'factory' in str.lower():
            str=str.replace(str,'Trades  Services')
    
    elif 'retail' in str.lower():
            str=str.replace(str,'Retail  Consumer Products')
    
    elif 'hr jobs' in str.lower():
            str=str.replace(str,'Human Resources  Recruitment')
    
    elif 'marketing' in str.lower():
            str=str.replace(str,'Marketing  Communications')
    
    elif 'estate' in str.lower():
            str=str.replace(str,'Real Estate  Property')
    
    elif 'customer' in str.lower():
            str=str.replace(str,'Call Centre  Customer Service')
    
    elif 'account' in str.lower():
            str=str.replace(str,'Accounting')

    elif 'health' in str.lower():
            str=str.replace(str,'Healthcare  Medical')
    
    elif 'logistics' in str.lower():
            str=str.replace(str,'Manufacturing Transport  Logistics')
    
    elif 'social' in str.lower():
            str=str.replace(str,'Community Services  Development')
                    
    elif 'bank' in str.lower() or 'finance' in str.lower():
            str=str.replace(str,'Banking  Financial Services')

    elif 'recruitment' in str.lower():
            str=str.replace(str,'Human Resources  Recruitment')
    
    elif 'engineering' in str.lower():
            str=str.replace(str,'Engineering')

    elif 'construction' in str.lower():
            str=str.replace(str,'Construction')
                    
    elif 'admin' in str.lower():
            str=str.replace(str,'Administration  Office Support')

    elif 'sales' in str.lower():
            str=str.replace(str,'Sales')
    
    elif 'it jobs' in str.lower():
            str=str.replace(str,'Information  Communication Technology')

    elif 'education' in str.lower():
            str=str.replace(str,'Education  Training')
    ###
    elif 'energy' in str.lower():
            str=str.replace(str,'Mining Resources  Energy')

    elif 'apprenticeships jobs' in str.lower():
            str=str.replace(str,'Apprenticeships, Training and Internships')
    
    elif 'training' in str.lower():
            str=str.replace(str,'Apprenticeships, Training and Internships')
    
    elif 'graduate' in str.lower():
           
            str=str.replace(str,'Apprenticeships, Training and Internships')

    elif 'charity' in str.lower():
            str=str.replace(str,'Community Services  Development')
    ##
    elif 'strategy' in str.lower():
            str=str.replace(str,'Consulting  Strategy')
    
    elif 'fmcg jobs' in str.lower():
            str=str.replace(str,'Retail  Consumer Products')

    elif 'leisure' in str.lower():
            str=str.replace(str,'Hospitality  Tourism')
    
    ##
    
    elif 'insurance' in str.lower():
            str=str.replace(str,'Insurance  Superannuation')

    elif 'media' in str.lower():
            str=str.replace(str,'Advertising Arts  Media')

    
    elif 'science' in str.lower():
            str=str.replace(str,'Science  Technology')

    
    elif 'motoring' in str.lower():
            str=str.replace(str,'Manufacturing Transport  Logistics')

    elif 'security' in str.lower():
            str=str.replace(str,'Trades  Services')

    str=str.replace('jobs',"")
    return re.sub(r'[^A-Za-z0-9 ]+','',str) 
    

### General cleaning function

In [10]:
def genclean(str):
    return re.sub(r'[^A-Za-z0-9 ]+','',str) 

## Clean Dataframe with cleaning functions applied to relevant attributes

In [11]:
clean_df=pd.DataFrame()

clean_df['job_title']=newdf['job_title'].apply(clean_titles)
clean_df['category']=newdf['category'].apply(clean_cat)
clean_df['state']=newdf['state'].apply(genclean)
clean_df['job_type']=newdf['job_type'].apply(genclean)
clean_df['geo']=newdf['geo'].apply(genclean)
clean_df['company_name']=newdf['company_name'].apply(genclean)
clean_df['city']=newdf['city'].apply(genclean)

clean_df['year']=newdf['new_post_date'].dt.year
clean_df['month']=newdf['new_post_date'].dt.month
clean_df['day']=newdf['new_post_date'].dt.day
clean_df['new_post_date']=newdf['new_post_date']
clean_df['month_name']=newdf['new_post_date'].dt.month_name()
clean_df['day_name']=newdf['new_post_date'].dt.day_name()


#### Location Dimension

In [12]:
df = clean_df[['geo','city','state']].drop_duplicates()
df['LOC_SK'] = range(1,len(df.index)+1)
print (df)

      geo                       city                           state  LOC_SK
0      AU                     Sydney   North Shore  Northern Beaches       1
1      AU                   Brisbane                Northern Suburbs       2
2      AU                     Sydney     Parramatta  Western Suburbs       3
3      AU                  Melbourne  Bayside  South Eastern Suburbs       4
4      AU                   Adelaide                      NOT LISTED       5
...    ..                        ...                             ...     ...
49788  uk             Skipton Bridge                 North Yorkshire    3432
49822  uk                   Tredegar                           Wales    3433
49870  uk                   Melbourn                   Hertfordshire    3434
49926  uk                 Somercotes                      Derbyshire    3435
49932  uk  Astmoor Industrial Estate                        Cheshire    3436

[3436 rows x 4 columns]


### Location Dimension to csv

In [13]:
df.to_csv('DimLocation.csv',columns=['LOC_SK','geo','state','city'],index=False,header=False)

#### Date Dimension

In [14]:
df1 = clean_df[['new_post_date','year','month_name','month','day_name','day']].drop_duplicates()
df1['DATE_SK'] = range(1,len(df1.index)+1)
print (df1)

      new_post_date  year month_name  month   day_name  day  DATE_SK
0        2018-04-15  2018      April      4     Sunday   15        1
3        2018-04-16  2018      April      4     Monday   16        2
4409     2018-04-17  2018      April      4    Tuesday   17        3
8127     2018-04-14  2018      April      4   Saturday   14        4
11389    2018-04-13  2018      April      4     Friday   13        5
...             ...   ...        ...    ...        ...  ...      ...
43956    2018-01-08  2018    January      1     Monday    8       70
47269    2017-12-28  2017   December     12   Thursday   28       71
48175    2018-01-10  2018    January      1  Wednesday   10       72
48634    2018-02-04  2018   February      2     Sunday    4       73
48767    2018-01-25  2018    January      1   Thursday   25       74

[74 rows x 7 columns]


### Date Dimension to csv

In [15]:
df1.to_csv('DimDate.csv',columns=['DATE_SK','new_post_date','year','month_name','month','day_name','day'],index=False,header=False)

#### Company Dimension

In [16]:
df2 = clean_df[['company_name','category']].drop_duplicates()
df2['COMP_SK'] = range(1,len(df2.index)+1)
print (df2)

                            company_name                   category  COMP_SK
0      Frontline Executive Retail Sydney  Retail  Consumer Products        1
1                              Powerlink        Government  Defence        2
2                    Richard Jay Laundry           Trades  Services        3
3                      Adaptalift Hyster           Trades  Services        4
4                      Bakers Delight GM           Trades  Services        5
...                                  ...                        ...      ...
49964          Jonathan Lee Recruitment        Consulting  Strategy    25617
49975                 Willis Global Ltd        Consulting  Strategy    25618
49987                             Elite                 purchasing     25619
49992             St Andrews Healthcare                 purchasing     25620
49998           Concept Human Solutions                 purchasing     25621

[25621 rows x 3 columns]


### Company Dimension to csv

In [17]:
df2.to_csv('DimCompany.csv',columns=['COMP_SK','category','company_name'],index=False,header=False)

#### Job Dimension

In [18]:
df3 = clean_df[['job_type','job_title']].drop_duplicates()
df3['JOB_SK'] = range(1,len(df3.index)+1)
print (df3)

                 job_type                           job_title  JOB_SK
0               Full Time                      Store Manager        1
1               Full Time             Client Solution Analyst       2
2               Full Time                 Service Technician        3
3               Full Time    Workshop Technician I Material H       4
4               Full Time         APPRENTICESHIP JUNIOR BAKER       5
...                   ...                                 ...     ...
49985  Permanent fulltime                    Head of Networks   42139
49987  Permanent fulltime  Logistics and Operations Assistant   42140
49992  Permanent fulltime            Strategic Sourcing Buyer   42141
49995   Contract fulltime             Scientist In vitro Cell   42142
49998  Permanent fulltime            AV Procurement Assistant   42143

[42143 rows x 3 columns]


### Job Dimension to csv

In [19]:
df3.to_csv('DimJob.csv',columns=['JOB_SK','job_type','job_title'],index=False,header=False)

## Merging Dimensions dataframes and original clean dataframe to obtain the fact table 

In [20]:
fact=pd.DataFrame()
new1_df = pd.merge(clean_df, df3,  how='left', left_on=['job_type','job_title'], right_on = ['job_type','job_title'])
new2_df = pd.merge(clean_df, df2,  how='left', left_on=['category','company_name'], right_on = ['category','company_name'])
new3_df = pd.merge(clean_df, df,  how='left', left_on=['geo','city','state'], right_on =['geo','city','state'])
new4_df = pd.merge(clean_df, df1,  how='left', left_on=['new_post_date','year','month_name','month','day_name','day'] ,right_on =['new_post_date','year','month_name','month','day_name','day'])
fact['JOB_SK']=new1_df['JOB_SK']
fact['COMP_SK']=new2_df['COMP_SK']
fact['LOC_SK']=new3_df['LOC_SK']
fact['DATE_SK']=new4_df['DATE_SK']

fact

,JOB_SK,COMP_SK,LOC_SK,DATE_SK
0,1,1,1,1
1,2,2,2,1
2,3,3,3,1
3,4,4,4,2
4,5,5,5,2
...,...,...,...,...
79995,42142,20355,153,17
79996,32094,21221,182,19
79997,29745,20092,200,54
79998,42143,25621,1030,16


### Fact table to csv

In [21]:
fact.to_csv('FactlessJob.csv',index=False,header=False)

In [22]:
##Optional
#clean_df.to_csv('clean.csv',index=False,header=True)